##          EJERCICIO 2
# “Discrimination in the rental housing market: A field experiment on the Internet” [Ahmed et al. (2008)]

---




## **Paquetes**

In [ ]:
install.packages("DeclareDesign")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Formula’, ‘Rcpp’, ‘RcppEigen’, ‘randomizr’, ‘fabricatr’, ‘estimatr’




In [ ]:
require("DeclareDesign")

In [ ]:
require("DesignLibrary")

In [ ]:
install.packages("margins")

In [ ]:
require(tidyverse)

In [ ]:
require("margins") # for margins
require(broom) # for tidy

In [ ]:
set.seed(1011)

# **1. Modelo**

In [ ]:
panels <- fabricate( ##### Genera la población y los efectos fijos
  listings = add_level(N = 12639, listing_fe = runif(N, -.2, .2)), ### efecto fijo de individuos
  hours = add_level(N = 3, hour_shock = runif(N, -.08, .08), nest = FALSE), ## efecto fijo hora de respuesta (1 a 1.5 hrs)
  obs = cross_levels(
    by = join_using(listings, hours),
    U =rnorm(N, 0, 0.2), #### Variacion aleatoria
    epsilon = listing_fe + hour_shock + U
  )


)

panels  %>% arrange(listings)


In [ ]:
design0 <- declare_model(
  panels, ### Poblacion creada
  rooms = sample(c(1,2,3,4,5,6,7,8), N, replace = TRUE), ### controles
  area = runif(N, min = 30, max = 120),
  rent_1000 = runif(N, min = 1, max = 100),
  m_area = rbinom(N, size=1, prob=0.448),
  company = rbinom(N, size=1, prob=0.096),
  female = rbinom(N, size=1, prob=0.364),
  immigrant = rbinom(N, size=1, prob=0.078),
  potential_outcomes(    ##### Resultados potenciales dejando fijo a Erik
    Y ~ rbinom(n = N, size = 1, prob =
      pmax(0, pmin(1, 0.126 * (Z == 1) - 0.261 * (Z == 2) +
        0.007 * rooms + 0.001 * area + 0.002 * rent_1000 +
        0.550 * company - 0.251 * m_area - 0.023 * female +
        0.227 * immigrant + epsilon)
      )),
    conditions = list(Z = c(0, 1, 2))
  )
) + NULL



head(draw_data(design0) %>% arrange(listings),100)

# **(A) Inquiry**

In [ ]:
design0 <- declare_model(
  panels,
  rooms = sample(c(1,2,3,4,5,6,7,8), N, replace = TRUE),
  area = runif(N, min = 30, max = 120),
  rent_1000 = runif(N, min = 1, max = 100),
  m_area = rbinom(N, size=1, prob=0.448),
  company = rbinom(N, size=1, prob=0.096),
  female = rbinom(N, size=1, prob=0.364),
  immigrant = rbinom(N, size=1, prob=0.078),
  potential_outcomes(
    Y ~ rbinom(n = N, size = 1, prob =
      pmax(0, pmin(1, 0.126 * (Z == 1) - 0.261 * (Z == 2) +
        0.007 * rooms + 0.001 * area + 0.002 * rent_1000 +
        0.550 * company - 0.251 * m_area - 0.023 * female +
        0.227 * immigrant + epsilon)
      )),
    conditions = list(Z = c(0, 1, 2))
  )
)+
declare_inquiry( ######### Tres medidas comparando los 3  grupos de interes
  ATE1 = mean(Y_Z_1 - Y_Z_0), #### Maria vs ERIK
  ATE2 = mean(Y_Z_2 - Y_Z_0), ### Mohamec vs ERIK
  ATE3 = mean(Y_Z_2 - Y_Z_1)) #### Mohamec vs Maria

# **(B) Data strategy**

In [ ]:
design0 <- declare_model(
  panels,
  rooms = sample(c(1,2,3,4,5,6,7,8), N, replace = TRUE),
  area = runif(N, min = 30, max = 120),
  rent_1000 = runif(N, min = 1, max = 100),
  m_area = rbinom(N, size=1, prob=0.448),
  company = rbinom(N, size=1, prob=0.096),
  female = rbinom(N, size=1, prob=0.364),
  immigrant = rbinom(N, size=1, prob=0.078),
  potential_outcomes(
    Y ~ rbinom(n = N, size = 1, prob =
      pmax(0, pmin(1, 0.126 * (Z == 1) - 0.261 * (Z == 2) +
        0.007 * rooms + 0.001 * area + 0.002 * rent_1000 +
        0.550 * company - 0.251 * m_area - 0.023 * female +
        0.227 * immigrant + epsilon)
      )),
    conditions = list(Z = c(0, 1, 2))
  )
)+
declare_inquiry(
  ATE1 = mean(Y_Z_1 - Y_Z_0),
  ATE2 = mean(Y_Z_2 - Y_Z_0),
  ATE3 = mean(Y_Z_2 - Y_Z_1))+
 declare_sampling(S = complete_rs(N = N, n = 500)) + #### Muestreo donde los 3 grupo eplican aleatorio
declare_assignment(Z = complete_ra(N, prob = 0.333333)) + ## balance de la asignación
declare_measurement(Y = reveal_outcomes(Y ~ Z))

# **(C) Answer Strategy**

In [ ]:
tidy_margins <- function(x) { ##### Funcion para los resultados marginales modelo probit
  tidy(margins(x, data = x$data), conf.int = TRUE)
}

In [ ]:
### Estimaciones probit (efectos marginales) de la probabilidad de ser invitado a más contactos y de ser invitado a proyecciones

design0 <- declare_model(
  panels,
  rooms = sample(c(1,2,3,4,5,6,7,8), N, replace = TRUE),
  area = runif(N, min = 30, max = 120),
  rent_1000 = runif(N, min = 1, max = 100),
  m_area = rbinom(N, size=1, prob=0.448),
  company = rbinom(N, size=1, prob=0.096),
  female = rbinom(N, size=1, prob=0.364),
  immigrant = rbinom(N, size=1, prob=0.078),
  potential_outcomes(
    Y ~ rbinom(n = N, size = 1, prob =
      pmax(0, pmin(1, 0.126 * (Z == 1) - 0.261 * (Z == 2) +
        0.007 * rooms + 0.001 * area + 0.002 * rent_1000 +
        0.550 * company - 0.251 * m_area - 0.023 * female +
        0.227 * immigrant + epsilon)
      )),
    conditions = list(Z = c(0, 1, 2))
  )
)+
declare_inquiry(
  ATE1 = mean(Y_Z_1 - Y_Z_0),
  ATE2 = mean(Y_Z_2 - Y_Z_0),
  ATE3 = mean(Y_Z_2 - Y_Z_1))+
declare_sampling(S = complete_rs(N = N, n = 500)) +
declare_assignment(Z = complete_ra(N, prob = 0.333333)) +
declare_measurement(Y = reveal_outcomes(Y ~ Z))+
declare_estimator(model = lm, formula = Y ~ Z, term = "Z", label = "OLS") + ###  Modelo Linea para comparar diferencias de medias
declare_estimator(model = glm, formula = Y ~ Z, family = "binomial", label = "Logit", .summary = tidy_margins) ### Modelo que mide el efecto marginal


In [ ]:
draw_estimates(design0)

# **Diagnosis**

In [ ]:
diagnose_design(design0, sims = 50) ### se verifica sesgo y poder con 50 corridas

# **heterogeneidad por origen de landlord (immigrante/native)**

In [ ]:
### Para  explorar la heterogeneidad se reaiza la interaccion entre migrante Mohame y Maria
#### Se toma las probabiidades de estas interacciones especiales reportadas en el paper
### No se tiene en cuenta la interaccion entre los otros controles
design1 <- declare_model(
  panels,
  rooms = sample(c(1,2,3,4,5,6,7,8), N, replace = TRUE),
  area = runif(N, min = 30, max = 120),
  rent_1000 = runif(N, min = 1, max = 100),
  m_area = rbinom(N, size=1, prob=0.448),
  company = rbinom(N, size=1, prob=0.096),
  female = rbinom(N, size=1, prob=0.364),
  immigrant = rbinom(N, size=1, prob=0.078),
  potential_outcomes(
    Y ~ rbinom(n = N, size = 1, prob =
      pmax(0, pmin(1, 0.126*(Z==1) - 0.261*(Z==2) +  0.036*(Z==1)*immigrant - 0.076*(Z==2)*immigrant +
        0.007*rooms + 0.001*area + 0.002*rent_1000 +  0.550*company - 0.251*m_area + 0.023*female +  0.227*immigrant))),
    conditions = list(Z = c(0, 1, 2))
  )
)+
declare_inquiry(
  ATE1 = mean(Y_Z_1 - Y_Z_0),
  ATE2 = mean(Y_Z_2 - Y_Z_0),
  ATE3 = mean(Y_Z_2 - Y_Z_1))+
  declare_sampling(S = complete_rs(N = N, n = 500)) +
  declare_assignment(Z = complete_ra(N, prob = 0.333333)) +
  declare_measurement(Y = reveal_outcomes(Y ~ Z)) +
  declare_estimator(model = lm, formula = Y ~ Z + immigrant + Z:immigrant, label = "OLS") +
  declare_estimator(model = glm, formula = Y ~ Z + immigrant + Z:immigrant, family = "binomial", label = "Logit", .summary = tidy_margins)

In [ ]:
diagnose_design(design1, sims = 50) ### se verifica sesgo y poder con 50 corridas de la nueva configuración

In [ ]:
draw_estimates(design1)